In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
from torch.utils.data import Dataset, DataLoader
import glob
import wandb
import os
import torch.optim as optimizers

In [3]:
import dfs_code
from torch_geometric.data import InMemoryDataset, Data
import pickle
import torch
import torch.nn as nn
import tqdm
import copy
import pandas as pd
#torch.multiprocessing.set_sharing_strategy('file_system') # this is important on local machine
#def set_worker_sharing_strategy(worker_id: int) -> None:
#    torch.multiprocessing.set_sharing_strategy('file_system')

In [4]:
import sys
sys.path = ['../../../src'] + sys.path
from dfs_transformer import EarlyStopping, DFSCodeSeq2SeqFC, smiles2graph

Using backend: pytorch
2021-09-09 11:22:29.659198: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/cuda/extras/CUPTI/lib64/:/opt/intel/lib:/opt/intel/mkl/lib/intel64:/opt/intel:/opt/ibm/ILOG/CPLEX_Studio1210/cplex/bin/x86-64_linux:/opt/ibm/ILOG/CPLEX_Studio1210/cplex/python/3.7/x86-64_linux:/opt/intel/clck_latest/lib:/opt/intel/daal/lib:/opt/intel/intelpython3/lib:/opt/intel/ipp/lib:/opt/intel/itac_2019/lib:/opt/intel/itac_latest/lib:/opt/intel/mkl/lib:/opt/intel/mkl_/lib:/opt/intel/mpirt/lib:/opt/intel/tbb/lib:/opt/intel/clck/2019.0/lib:/opt/intel/compilers_and_libraries_2019/linux/lib:/opt/intel/compilers_and_libraries/linux/lib:/opt/intel/itac/2019.0.018/lib:/opt/intel/itac_2019/intel64/lib:/opt/intel/itac_latest/intel64/lib:/opt/intel/parallel_studio_xe_2019.0.045/clck_2019/lib:/opt/intel/parallel_studio_xe

In [5]:
import dgl
import deepchem as dc

In [6]:
wandb.init(project='pubchem-mini', entity='chrisxx', name='gnn-10k')
config = wandb.config
config.fraction_missing = 0.25
config.n_atoms = 118
config.n_bonds = 5
config.node_out_feats = 64
config.edge_hidden_feats = 128
config.number_atom_features = 133
config.number_bond_features = 14
config.num_step_message_passing = 6
config.max_nodes = 250
config.max_edges = 500
config.n_files = 10
config.n_splits = 1
config.n_iter_per_split = 10000
config.lr = 0.003
config.n_epochs = 10000
config.lr_adjustment_period = 200
config.patience = 5
config.factor = 0.96
config.minimal_lr = 6e-8
config.batch_size = 125
config.accumulate_grads = 1
config.valid_patience = 10000
config.valid_minimal_improvement=0.00
config.model_dir = "../../../models/pubchem_mini/gnn6/"
config.data_dir = "/mnt/project/pubchem_noH/"
config.data_dir = "../../../results/pubchem/mini10k/"
config.pretrained_dir = None#"../../models/chembl/better_transformer/medium/"
config.num_workers = 0
config.prefetch_factor = 2
config.persistent_workers = False
config.load_last = False


ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: chrisxx (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
2021-09-09 11:22:32.489581: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/cuda/extras/CUPTI/lib64/:/opt/intel/lib:/opt/intel/mkl/lib/intel64:/opt/intel:/opt/ibm/ILOG/CPLEX_Studio1210/cplex/bin/x86-64_linux:/opt/ibm/ILOG/CPLEX_Studio1210/cplex/python/3.7/x86-64_linux:/opt/intel/clck_latest/lib:/opt/intel/daal/lib:/opt/intel/intelpython3/lib:/opt/intel/ipp/lib:/opt/intel/itac_2019/lib:/opt/intel/itac_latest/lib:/opt/intel/mkl/lib:/opt/intel/mkl_/lib:/opt/intel/mpirt

In [7]:
os.makedirs(config.model_dir, exist_ok=True)

In [8]:
path = config.data_dir

In [9]:
class PubChem(Dataset):
    """PubChem dataset of molecules and minimal DFS codes."""
    def __init__(self, path=path, n_used = 8, n_splits = 64, max_nodes=config.max_nodes,
                 max_edges=config.max_edges, useHs=False, addLoops=False, memoryEfficient=False,
                 transform=None, n_mols_per_dataset=np.inf, node_feature_key='x', edge_feature_key='edge_attr'):
        self.path = path
        self.data = []
        self.path = path
        self.n_used = n_used
        self.n_splits = n_splits
        self.useHs = useHs
        self.addLoops = addLoops
        self.memoryEfficient = memoryEfficient
        self.n_mols_per_dataset = n_mols_per_dataset
        self.max_nodes = max_nodes
        self.max_edges = max_edges
        self.node_feature_key = node_feature_key
        self.edge_feature_key = edge_feature_key
        self.prepare()
        
        
    def prepare(self):
        codes_all = {}
        d_all = {}
        i2didx = {}
        perm = np.random.permutation(self.n_splits)
        for i in tqdm.tqdm(perm[:self.n_used]):
            dname = glob.glob(self.path+"/%d/min_dfs_codes_split*.json"%(i+1))[0]
            didx = int(dname.split("split")[-1][:-5])
            dname2 = self.path+"/%d/data_split%d.json"%(i+1, didx)
            with open(dname, 'r') as f:
                codes = json.load(f)
                for key, val in codes.items():
                    codes_all[key] = val
            with open(dname2, 'r') as f:
                d_dict = json.load(f)
                for key, val in d_dict.items():
                    d_all[key] = val
        
        for smiles, code in tqdm.tqdm(codes_all.items()):
            if len(self.data) > self.n_mols_per_dataset:
                break
            if code['min_dfs_code'] is not None and len(code['min_dfs_code']) > 1:
                d = d_all[smiles]
                #d = smiles2graph(smiles, useHs=self.useHs, addLoops=self.addLoops, skipCliqueCheck=False)
                if len(d['z']) > self.max_nodes:
                    continue
                if len(d['edge_attr']) > 2*self.max_edges:
                    continue
                
                
                z = torch.tensor(d['z'], dtype=torch.long)
                x = torch.tensor(d['x'], dtype=torch.float32)
                
                
                data_ = Data(x=x,
                             z=z,
                             edge_attr=torch.tensor(d['edge_attr']),
                             edge_index=torch.tensor(d['edge_index'], dtype=torch.long),
                             min_dfs_code=torch.tensor(code['min_dfs_code']),
                             min_dfs_index=torch.tensor(code['dfs_index'], dtype=torch.long),
                             smiles=smiles,
                             node_features=torch.tensor(d['atom_features'], dtype=torch.float32),
                             edge_features=torch.tensor(d['bond_features'], dtype=torch.float32))
                self.data += [data_]   
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [10]:
def collate_fn(dlist):
    dglbatch = []
    for d in dlist:
        g = dgl.graph((d.edge_index[0], d.edge_index[1]))
        dfs_ind = nn.functional.one_hot(d.min_dfs_index, num_classes=config.max_nodes+1).float()
        n = len(dfs_ind)
        missing_idx = np.random.permutation(n)[:int(config.fraction_missing*n)]
        dfs_ind[missing_idx] = 0
        dfs_ind[missing_idx, -1] = 1
        g.ndata['nf'] = torch.cat((d.node_features, dfs_ind), dim=1)
        g.ndata['dfs'] = d.min_dfs_index
        g.edata['ef'] = d.edge_features
        dglbatch += [g]
    dglbatch = dgl.batch(dglbatch)
    return dglbatch, dglbatch.ndata['nf'], dglbatch.edata['ef'], dglbatch.ndata['dfs']

In [11]:
ngpu=1
device = torch.device('cuda:0' if (torch.cuda.is_available() and ngpu > 0) else 'cpu')

In [12]:
to_cuda = lambda T: [t.to(device) for t in T]

In [13]:
classifier = dc.models.torch_models.MPNN(1, mode='classification', 
                                    node_out_feats=config.node_out_feats,
                                    edge_hidden_feats=config.edge_hidden_feats,
                                    #add the dfs index information to the input
                                    number_atom_features=config.number_atom_features + (config.max_nodes + 1),
                                    number_bond_features=config.number_bond_features, 
                                    num_step_message_passing=config.num_step_message_passing,
                                    nfeat_name='node_features', 
                                    efeat_name='edge_features')

model = classifier.model.gnn

In [14]:
if config.pretrained_dir is not None:
    model.load_state_dict(torch.load(config.pretrained_dir+'checkpoint.pt'))

In [15]:
if config.load_last:
    model.load_state_dict(torch.load(config.model_dir+'checkpoint.pt'))

In [16]:
model_head = nn.Linear(config.node_out_feats, config.max_nodes)

In [17]:
optim = optimizers.Adam(list(model.parameters())+list(model_head.parameters()), lr=config.lr)

lr_scheduler = optimizers.lr_scheduler.ReduceLROnPlateau(optim, mode='min', verbose=True, patience=config.patience, factor=config.factor)
#lr_scheduler = optimizers.lr_scheduler.ExponentialLR(optim, gamma=config.factor)

early_stopping = EarlyStopping(patience=config.valid_patience, delta=config.valid_minimal_improvement,
                              path=config.model_dir+'checkpoint.pt')
bce = torch.nn.BCEWithLogitsLoss()
ce = torch.nn.CrossEntropyLoss(ignore_index=-1)
softmax = nn.Softmax(dim=2)

In [18]:
model.to(device)
model_head.to(device)
print()

In [ ]:
try:
    for epoch in range(config.n_epochs):
        epoch_loss = 0
        epoch_acc = 0
        for split in range(config.n_splits):
            
            n_ids = config.n_files//config.n_splits
            dataset = PubChem(n_used = n_ids, n_splits = config.n_files)
            loader = DataLoader(dataset, batch_size=config.batch_size, shuffle=True, pin_memory=False, 
                                collate_fn=collate_fn,
                   num_workers=config.num_workers, prefetch_factor=config.prefetch_factor, 
                    persistent_workers=config.persistent_workers)
            for j in range(config.n_iter_per_split):
                pbar = tqdm.tqdm(loader)
                for i, data in enumerate(pbar):
                    if i % config.accumulate_grads == 0: #bei 0 wollen wir das
                        optim.zero_grad()
                    g, ndata, edata, labels = data
                    g = g.to(device)
                    ndata = ndata.to(device)
                    edata = edata.to(device)
                    labels = labels.to(device)
                    
                    #prediction
                    features = model(g, ndata, edata)
                    pred = model_head(features)
                    loss = ce(pred, labels)
                    acc = (torch.argmax(pred, axis=1) == labels).sum()/len(labels)
                    loss.backward()
                    #torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
                    if (i+1) % config.accumulate_grads == 0:
                        optim.step() # bei 2 wollen wir das
                    epoch_loss = (epoch_loss*i + loss.item())/(i+1)
                    epoch_acc = (epoch_acc*i + acc.item())/(i+1)

                    curr_lr = list(optim.param_groups)[0]['lr']
                    wandb.log({'loss':epoch_loss, 'acc':epoch_acc,'learning rate':curr_lr})
                    pbar.set_description('Epoch %d: CE %2.6f, acc %2.6f'%(epoch+1, epoch_loss, epoch_acc))

                    if i % config.lr_adjustment_period == 0:
                        early_stopping(epoch_loss, model)
                        lr_scheduler.step(epoch_loss)
                        if early_stopping.early_stop:
                            break

                        if curr_lr < config.minimal_lr:
                            break
                
                
            del dataset
            del loader
        

except KeyboardInterrupt:
    torch.save(model.state_dict(), config.model_dir+'_keyboardinterrupt.pt')
    print('keyboard interrupt caught')


Epoch 1: CE 0.280951, acc 0.928542:  38%|███████████████████████████████████████████████████▊                                                                                      | 30/80 [00:03<00:05,  8.87it/s]

In [ ]:
data[1].shape

In [ ]:
data[1][:, -1].sum()

In [ ]:
len(data[1])